<a href="https://colab.research.google.com/github/HomayounfarM/Classification/blob/main/Logistic_regression_Ln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Logistic regression is a statistical method used for modeling a binary dependent variable, meaning the outcome has two possible values (e.g., success/failure, yes/no, high/low). It is widely used in various fields such as medicine, social sciences, and machine learning for classification tasks.

To illustrate how the logistic regression works behind the scenes, I walk you through an example defining all required functions. We also do the same using the ski-learning package.
Assume we study the effect of the phosphorous and its interaction with the soil pH on yield in a bean experiment. we have a dataset including
 the M3-P soil extractant (continuous variable), soil pH (continuous variable) and the yield (categorized as 'high' or 'low'). For the yield, 1 means ‘high’ and 0 means ‘low’.


In [ ]:
# Install libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import math
import io
import ssl
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

In [ ]:
# Reading the dataset from GITHUB
ssl._create_default_https_context = ssl._create_unverified_context
url = "https://raw.githubusercontent.com/HomayounfarM/Classification/main/data/ex2data2.csv"
df = pd.read_csv(url)
df.columns = ['pH', 'M3_P', 'Yield']
df['pH_r'] = (df['pH']-np.mean(df['pH']))/np.std(df['pH'])
df['M3_P_r'] = (df['M3_P']-np.mean(df['M3_P']))/np.std(df['M3_P'])


df.head(5)


In [ ]:
# load dataset
X_scaled = np.array(df[['pH_r', 'M3_P_r']])
YY = np.array(df['Yield'])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, YY, test_size=0.2, random_state=42)

In [ ]:
x1 = df['pH_r']
x2 = df['M3_P_r']
y = df['Yield']


plt.scatter(x1[y==0], x2[y==0], s=3, c='r')
plt.scatter(x1[y==1], x2[y==1], s=3, c='b')

# Add axis labels
plt.xlabel('pH_r')
plt.ylabel('M3_P_r')

# Add a legend
plt.legend(['Low yield', 'High yield'])

#Sigmoid function

In [ ]:
# Sigmoid function
def sigmoid(z):

    g = 1/(1+np.exp(-z))

    return g

In [ ]:
print ("sigmoid(0) = " + str(sigmoid(0)))

In [ ]:
# Cost function
def compute_cost(X, y, w, b):

    m, n = X.shape

    z = X.dot(w)+b
    f = sigmoid(z)
    total_cost = (1/m) * (-y.T.dot(np.log(f))- (1-y).T.dot(np.log(1-f)))

    return total_cost

In [ ]:
m, n = X_train.shape

# Compute and display cost with w initialized to zeroes
initial_w = np.zeros(n)
initial_b = 0.
cost = compute_cost(X_train, y_train, initial_w, initial_b)
print('Cost at initial w (zeros): {:.3f}'.format(cost))

In [ ]:
# Gradient descent
def compute_gradient(X, y, w, b):

    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.

    y = y.reshape(-1,1)
    w = w.reshape(-1,1)

    f = sigmoid(X.dot(w)+b)
    diff_fy = f-y
    dj_dww = (1/m) * diff_fy.T.dot(X)
    dj_dww = dj_dww.T
    dj_dw = np.reshape(dj_dww, -1)
    dj_db = (1/m) * (dj_db + np.sum(diff_fy))

    return dj_db, dj_dw

In [ ]:
# Compute and display gradient with w initialized to zeroes
initial_w = np.zeros(n)
initial_b = 0.

dj_db, dj_dw = compute_gradient(X_train, y_train, initial_w, initial_b)
print(f'dj_db at initial w (zeros):{dj_db}' )
print(f'dj_dw at initial w (zeros):{dj_dw.tolist()}' )

In [ ]:
# Compute and display cost and gradient with non-zero w
test_w = np.array([ 0.2, -0.5])
test_b = -24
dj_db, dj_dw  = compute_gradient(X_train, y_train, test_w, test_b)

print('dj_db at test_w:', dj_db)
print('dj_dw at test_w:', dj_dw.tolist())


In [ ]:
# Performs batch gradient descent to learn theta. Updates theta by taking num_iters gradient steps with learning rate alpha
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters):

    # number of training examples
    m = len(X)

    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w_history = []

    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in)

        # Update Parameters using w, b, alpha and gradient
        w_in = w_in - alpha * dj_dw
        b_in = b_in - alpha * dj_db

        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion
            cost =  cost_function(X, y, w_in, b_in)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")

    return w_in, b_in, J_history, w_history #return w and J,w history for graphing

In [ ]:
np.random.seed(1)
intial_w = 0.01 * (np.random.rand(2).reshape(-1,1) - 0.5)
initial_b = -8


# Some gradient descent settings
iterations = 100000
alpha = 0.7

w,b, J_history,_ = gradient_descent(X_train ,y_train, initial_w, initial_b,
                                   compute_cost, compute_gradient, alpha, iterations)
intial_w, initial_b, w,b

In [ ]:
x_temp = np.linspace(min(x1), max(x1), num=len(x1))
y_predicted = -(w[1]*x_temp+b)/w[0]

plt.plot(x_temp, y_predicted, color='purple')

-w[1]/w[0], -b/w[0], -intial_w[1]/intial_w[0], -initial_b/intial_w[0], intial_w[0],intial_w[1],initial_b

#Plotting the decision boundary

Try to plot decision boundary from got in previous step.

In [ ]:
x1 = df['pH_r']
x2 = df['M3_P_r']
y = df['Yield']

plt.scatter(x1[y==0], x2[y==0], s=3, c='r')
plt.scatter(x1[y==1], x2[y==1], s=3, c='b')

#add line to show fitted polynomial regression model
plt.plot(x_temp, y_predicted, color='purple')

# Add axis labels
plt.xlabel('pH_r')
plt.ylabel('M3_P_r')

# Add a legend
plt.legend(['Low yield', 'High yield'])

In [ ]:
# prediction function
def predict(X, w, b):

    # number of training examples
    m, n = X.shape
    p = np.zeros(m)

    z = X.dot(w)+b
    f = sigmoid(z)
    p = f >= 0.5

    return p

In [ ]:
# Test your predict code
np.random.seed(1)
tmp_w = np.random.randn(2)
tmp_b = 0.3
tmp_X = np.random.randn(50, 2) - 0.5

tmp_p = predict(tmp_X, tmp_w, tmp_b)
print(f'Output of predict: shape {tmp_p.shape}, value {tmp_p}')

In [ ]:
#Compute accuracy on our training set
p = predict(X_train, w,b)
print('Train Accuracy: %f'%(np.mean(p == y_train) * 100))

# Using skit-learn

In [ ]:
# load dataset
XX = np.array(df[['pH', 'M3_P']])
YY = np.array(df['Yield'])

#Feature scaling
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
X_scaled = scale.fit_transform(XX)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, YY, test_size=0.2, random_state=42)

# Initialize the logistic regression model
model = LogisticRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = model.predict(X_test)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)

# Extract the coefficients
coefficients = model.coef_[0]
intercept = model.intercept_[0]
coefficients, intercept

#Plotting the decision boundary
x1 = X_scaled[:,0]
x2 = X_scaled[:,1]
y = YY

plt.scatter(x1[y==0], x2[y==0], s=3, c='r')
plt.scatter(x1[y==1], x2[y==1], s=3, c='b')

#add line to show fitted polynomial regression model
x_temp = np.linspace(min(x1), max(x1), num=len(x1))
y_predicted = -(coefficients[1]*x_temp+intercept)/coefficients[0]

plt.plot(x_temp, y_predicted, color='purple')

# Add axis labels
plt.xlabel('pH_r')
plt.ylabel('M3_P_r')

# Add a legend
plt.legend(['Low yield', 'High yield'])

# Regularization:

Regularization is a technique used to prevent overfitting.
Indeed, there are different ways to handle overfitting as follows:

1- collecting more data: sometimes, this option is not working.

2- Feature selection. Selecting the most effective features.

3- Reduce the size of the parameters or regularize the parameters.

Here, I explain how regularization works in a logistic regression.

Regularization is typically adding a penalty term to the loss function, which penalizes the coefficients of the model.

There are two common types of regularization:

1- L1 Regularization (Lasso Regularization):

It  adds a penalty proportional to the absolute value of the coefficients leading some coefficients to be exactly zero, effectively performing feature selection.

2- L2 Regularization (Ridge Regularization):

It adds a penalty proportional to the square of the coefficients leading to small but non-zero coefficients.

In [ ]:
# Reading the dataset from GITHUB
ssl._create_default_https_context = ssl._create_unverified_context
url = "https://raw.githubusercontent.com/HomayounfarM/Classification/main/data/ex2data2.csv"
df = pd.read_csv(url)
df.columns = ['pH', 'M3_P', 'Yield']
df['pH_r'] = (df['pH']-np.mean(df['pH']))/np.std(df['pH'])
df['M3_P_r'] = (df['M3_P']-np.mean(df['M3_P']))/np.std(df['M3_P'])


df.head(5)

In [ ]:
# load dataset
X_scaled = np.array(df[['pH_r', 'M3_P_r']])
YY = np.array(df['Yield'])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, YY, test_size=0.2, random_state=42)

In [ ]:
x1 = df['pH_r']
x2 = df['M3_P_r']
y = df['Yield']


plt.scatter(x1[y==0], x2[y==0], s=3, c='r')
plt.scatter(x1[y==1], x2[y==1], s=3, c='b')

# Add axis labels
plt.xlabel('pH_r')
plt.ylabel('M3_P_r')

# Add a legend
plt.legend(['Low yield', 'High yield'])

In [ ]:
def map_feature(X, Y):
  # demonstrate the types of features created
  import numpy as np
  from sklearn.preprocessing import PolynomialFeatures
  # define the dataset
  data = np.vstack([X,Y]).T
  # perform a polynomial features transform of the dataset
  poly = PolynomialFeatures(degree=2, include_bias=False)
  data = poly.fit_transform(data)
  return data

In [ ]:
print("Original shape of data:", X_train.shape)

mapped_X =  map_feature(X_train[:, 0], X_train[:, 1])
print("Shape after feature mapping:", mapped_X.shape)

In [ ]:
print("X_train[0]:", X_train[0])
print("mapped X_train[0]:", mapped_X[0])

In [ ]:
# Sigmoid function
def sigmoid(z):

    g = 1/(1+np.exp(-z))

    return g

In [ ]:
# Cost function
def compute_cost(X, y, w, b, lambda_= 1):

    m, n = X.shape

    z = X.dot(w)+b
    f = sigmoid(z)
    total_cost = (1/m) * (-y.T.dot(np.log(f))- (1-y).T.dot(np.log(1-f)))

    return total_cost

In [ ]:
# Gradient descent
def compute_gradient(X, y, w, b, lambda_=None):

    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.

    y = y.reshape(-1,1)
    w = w.reshape(-1,1)

    f = sigmoid(X.dot(w)+b)
    diff_fy = f-y
    dj_dww = (1/m) * diff_fy.T.dot(X)
    dj_dww = dj_dww.T
    dj_dw = np.reshape(dj_dww, -1)
    dj_db = (1/m) * (dj_db + np.sum(diff_fy))

    return dj_db, dj_dw

In [ ]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_):
    """
    Performs batch gradient descent to learn theta. Updates theta by taking
    num_iters gradient steps with learning rate alpha

    Args:
      X :    (array_like Shape (m, n)
      y :    (array_like Shape (m,))
      w_in : (array_like Shape (n,))  Initial values of parameters of the model
      b_in : (scalar)                 Initial value of parameter of the model
      cost_function:                  function to compute cost
      alpha : (float)                 Learning rate
      num_iters : (int)               number of iterations to run gradient descent
      lambda_ (scalar, float)         regularization constant

    Returns:
      w : (array_like Shape (n,)) Updated values of parameters of the model after
          running gradient descent
      b : (scalar)                Updated value of parameter of the model after
          running gradient descent
    """

    # number of training examples
    m = len(X)

    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w_history = []

    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)

        # Update Parameters using w, b, alpha and gradient
        w_in = w_in - alpha * dj_dw
        b_in = b_in - alpha * dj_db

        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion
            cost =  cost_function(X, y, w_in, b_in, lambda_)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")

    return w_in, b_in, J_history, w_history #return w and J,w history for graphing

In [ ]:
def compute_cost_reg(X, y, w, b, lambda_ = 1):
    """
    Computes the cost over all examples
    Args:
      X : (array_like Shape (m,n)) data, m examples by n features
      y : (array_like Shape (m,)) target value
      w : (array_like Shape (n,)) Values of parameters of the model
      b : (array_like Shape (n,)) Values of bias parameter of the model
      lambda_ : (scalar, float)    Controls amount of regularization
    Returns:
      total_cost: (scalar)         cost
    """

    m, n = X.shape

    # Calls the compute_cost function that you implemented above
    cost_without_reg = compute_cost(X, y, w, b)

    # You need to calculate this value
    reg_cost = 0.

    ### START CODE HERE ###

    reg_cost = np.sum(w**2)
    ### END CODE HERE ###

    # Add the regularization cost to get the total cost
    total_cost = cost_without_reg + (lambda_/(2 * m)) * reg_cost

    return total_cost

In [ ]:
def compute_gradient_reg(X, y, w, b, lambda_ = 1):
    """
    Computes the gradient for linear regression

    Args:
      X : (ndarray Shape (m,n))   variable such as house size
      y : (ndarray Shape (m,))    actual value
      w : (ndarray Shape (n,))    values of parameters of the model
      b : (scalar)                value of parameter of the model
      lambda_ : (scalar,float)    regularization constant
    Returns
      dj_db: (scalar)             The gradient of the cost w.r.t. the parameter b.
      dj_dw: (ndarray Shape (n,)) The gradient of the cost w.r.t. the parameters w.

    """
    m, n = X.shape

    dj_db, dj_dw = compute_gradient(X, y, w, b)

    ### START CODE HERE ###

    dj_dw = dj_dw + (lambda_/m) * w

    ### END CODE HERE ###

    return dj_db, dj_dw

In [ ]:
# Compute and display cost with w initialized to zeroes
X_mapped = map_feature(X_train[:, 0], X_train[:, 1])

np.random.seed(1)
initial_w = 0.01 * (np.random.rand(X_mapped.shape[1]) - 0.5)
initial_b = -8

lambda_ = 0.5

cost = compute_cost_reg(X_mapped, y_train, initial_w, initial_b, lambda_)
print("Regularized cost :", cost)

In [ ]:
X_mapped = map_feature(X_train[:, 0], X_train[:, 1])

np.random.seed(1)
initial_w = 0.01 * (np.random.rand(X_mapped.shape[1]) - 0.5)
initial_b = -8

lambda_ = 0.05

# Some gradient descent settings
iterations = 100000
alpha = 0.01

w,b, J_history,_ = gradient_descent(X_mapped, y_train, initial_w, initial_b,
                                    compute_cost_reg, compute_gradient_reg,
                                    alpha, iterations, lambda_)
initial_w, initial_b, w,b


#Plotting the decision boundary

Try to plot decision boundary from got in previous step.

In [ ]:
#Compute accuracy on the training set
p = predict(X_mapped, w, b)

print('Train Accuracy: %f'%(np.mean(p == y_train) * 100))

In [ ]:
x_temp = np.linspace(min(x1), max(x1), num=len(x1))
y_predicted = -(w[1]*x_temp+b)/w[0]


part1 = -(w[1]+w[4]*x_temp)/(2*w[3])
discriminant = (w[1]+w[4]*x_temp)**2-4*w[3]*(x_temp*w[0]+x_temp**2*w[2]+b)

# Check if there are real roots
if discriminant.any() >= 0:
    # Calculate the roots using the quadratic formula
    root1 = (part1 + np.sqrt(discriminant)) / (2 * w[2])
    root2 = (part1 - np.sqrt(discriminant)) / (2 * w[2])



In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

df_temp = pd.DataFrame(w*X_mapped+b)
df_temp.head(5)
y_temp = np.zeros(len(df_temp))

# Fit linear regression model
model = LinearRegression()
model.fit(df_temp, y_temp)

x_temp = np.linspace(min(x1), max(x1), num=len(x1))
y_pred = model.predict(x_temp)

In [ ]:
x1 = df['pH_r']
x2 = df['M3_P_r']
y = df['Yield']


plt.scatter(x1[y==0], x2[y==0], s=3, c='r')
plt.scatter(x1[y==1], x2[y==1], s=3, c='b')

#add line to show fitted polynomial regression model
plt.plot(x_temp,root2, color='purple')

# Add axis labels
plt.xlabel('pH_r')
plt.ylabel('M3_P_r')

# Add a legend
plt.legend(['Low yield', 'High yield'])

# Using Skit-Learn



In [ ]:
def map_feature(X, Y):
  # demonstrate the types of features created
  import numpy as np
  from sklearn.preprocessing import PolynomialFeatures
  # define the dataset
  data = np.vstack([X,Y]).T
  # perform a polynomial features transform of the dataset
  poly = PolynomialFeatures(degree=2, include_bias=False)
  data = poly.fit_transform(data)
  return data

In [ ]:
# load dataset
XX = np.array(df[['pH', 'M3_P']])
YY = np.array(df['Yield'])

#Feature scaling
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
X_scaled = scale.fit_transform(XX)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, YY, test_size=0.2, random_state=42)

X_mapped_train = map_feature(X_train[:, 0], X_train[:, 1])
X_mapped_test = map_feature(X_test[:, 0], X_test[:, 1])

# Initialize the logistic regression model
# C parameter controls the regularization strength (inverse of regularization strength)
# Higher C means less regularization (default C=1.0)
model = LogisticRegression(penalty='l2', C=15, solver='lbfgs', max_iter=10000, random_state=1)


# Fit the model on the training data
model.fit(X_mapped_train, y_train)

# Predict the labels for the test set
y_pred = model.predict(X_mapped_test)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)

# Extract the coefficients
coefficients = model.coef_[0]
intercept = model.intercept_[0]
coefficients, intercept

#Plotting the decision boundary
x1 = X_scaled[:,0]
x2 = X_scaled[:,1]
y = YY

plt.scatter(x1[y==0], x2[y==0], s=3, c='r')
plt.scatter(x1[y==1], x2[y==1], s=3, c='b')

#add line to show fitted polynomial regression model
x_temp = np.linspace(min(x1), max(x1), num=len(x1))
y_predicted = -(coefficients[1]*x_temp+intercept)/coefficients[0]

# Deriving the relationship between x1 and x2
part1 = -(coefficients[1]+coefficients[4]*x_temp)/(2*coefficients[3])
discriminant = (coefficients[1]+coefficients[4]*x_temp)**2-4*coefficients[3]*(x_temp*coefficients[0]+x_temp**2*coefficients[2]+intercept)
# Check if there are real roots
if discriminant.any() >= 0:
    # Calculate the roots using the quadratic formula
    root1 = (part1 + np.sqrt(discriminant)) / (2 * w[2])
    root2 = (part1 - np.sqrt(discriminant)) / (2 * w[2])

plt.plot(x_temp, root2, color='purple')

# Add axis labels
plt.xlabel('pH_r')
plt.ylabel('M3_P_r')

# Add a legend
plt.legend(['Low yield', 'High yield'])
coefficients

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def PolyCoefficients(x, coeffs):
    """ Returns a polynomial for ``x`` values for the ``coeffs`` provided.

    The coefficients must be in ascending order (``x**0`` to ``x**o``).
    """
    o = len(coeffs)
    print(f'# This is a polynomial of order {o}.')
    y = 0
    for i in range(o):
        y += coeffs[i]*x**i
    return y

x = np.linspace(0, 9, 10)
coeffs = [1, 2, 3, 4, 5]
plt.plot(x, PolyCoefficients(x, coeffs))
plt.show()